# Tigrigna Real-time Spell Checker

This notebook creates an enhanced Tigrigna keyboard with real-time spell checking as you type!

In [13]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re
import os

## 1. Load the Tigrigna Dictionary

In [14]:
def load_dictionary(file_path='tigrigna_dictionary.txt'):
    """Load the Tigrigna dictionary"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            dictionary = {line.strip() for line in file if line.strip()}
        return dictionary
    except FileNotFoundError:
        print(f"Error: Dictionary file '{file_path}' not found.")
        return set()
    except Exception as e:
        print(f"Error loading dictionary: {e}")
        return set()

# Load the dictionary
dictionary = load_dictionary()
print(f"Loaded {len(dictionary)} words from the dictionary")

Loaded 294 words from the dictionary


## 2. Helper Functions for Spell Checking

In [15]:
def levenshtein_distance(s1, s2):
    """Calculate the Levenshtein distance between two strings"""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def get_suggestions(word, dictionary, max_distance=2, max_suggestions=5):
    """Generate suggestions for a misspelled word"""
    suggestions = []
    
    for dict_word in dictionary:
        distance = levenshtein_distance(word, dict_word)
        if distance <= max_distance:
            suggestions.append((dict_word, distance))
    
    suggestions.sort(key=lambda x: x[1])
    return [word for word, _ in suggestions[:max_suggestions]]

def generate_variants(base_char):
    """Generate vowel variants for a Tigrigna character"""
    if base_char == ' ' or not base_char or len(base_char) != 1:
        return [base_char]
    
    try:
        base_code = ord(base_char)
        variants = []
        for i in range(8):  # Include the 8th form if it exists (base+7)
            try:
                variant_code = base_code + i
                if 0xD800 <= variant_code <= 0xDFFF or variant_code > 0x10FFFF:
                    # Skip invalid Unicode code points
                    continue
                    
                variant = chr(variant_code)
                variants.append(variant)
            except:
                # If this variant doesn't exist, skip it
                pass
        
        return variants if variants else [base_char]
    except:
        return [base_char]

## 3. Real-time Spell Checking

In [16]:
def create_real_time_keyboard():
    """
    Create the Tigrigna keyboard with real-time spell checking
    """
    # Create the text area for input
    text_area = widgets.Textarea(
        placeholder='ኣብዚ ይጻሓፉ...',  # Type here in Tigrigna
        layout=widgets.Layout(width='100%', height='100px', 
                             border='1px solid #ddd', 
                             padding='10px')
    )
    
    # Create the live spell check display area
    spell_check_display = widgets.HTML(
        value='<div id="spell-check-display" style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 50px;">Spell check results will appear here as you type...</div>',
        layout=widgets.Layout(width='100%')
    )
    
    # Create variants container (for the green buttons)
    variants_container = widgets.HBox([], 
                                     layout=widgets.Layout(
                                         width='100%', 
                                         background_color='#000000',
                                         padding='5px'))
    
    # Keep track of the currently selected base character
    current_base = [None]
    
    # Define the Tigrigna keyboard layout
    # First row (numbers)
    row1 = ['፩', '፪', '፫', '፬', '፭', '፮', '፯', '፰', '፱', '፲']
    
    # Second row (first row of Tigrigna characters)
    row2 = ['ሀ', 'ሐ', 'ሠ', 'ረ', 'ሰ', 'ሸ', 'ቀ', 'ቐ', 'በ', 'ቨ']
    
    # Third row (second row of Tigrigna characters)
    row3 = ['ተ', 'ቸ', 'ኀ', 'ነ', 'ኘ', 'አ', 'ከ', 'ኸ', 'ወ', 'ዐ']
    
    # Fourth row (third row of Tigrigna characters) 
    row4 = ['ዘ', 'ژ', 'የ', 'ደ', 'ጀ', 'ገ', 'ጠ', 'ጨ', 'ጰ', 'ጸ']
    
    # Fifth row (fourth row of Tigrigna characters)
    row5 = ['ፀ', 'ፈ', 'ፐ', 'ቦ', 'ቱ', 'ሙ', 'ሉ', 'ኢ', 'ኣ', 'ኡ']
    
    # Create row structure for the main keyboard
    keyboard_layout = [row1, row2, row3, row4, row5]
    
    # Create the main keyboard container with black background
    keyboard_container = widgets.VBox([], 
                                     layout=widgets.Layout(
                                         width='100%',
                                         background_color='#000000',
                                         padding='5px'))
    
    # Create a label for the keyboard
    keyboard_label = widgets.HTML(
        value='<div style="text-align: center; color: white; padding: 5px;">Tigrigna</div>',
        layout=widgets.Layout(width='100%')
    )
    
    # Function to create a character button
    def create_char_button(char, width='34px'):
        btn = widgets.Button(
            description=char,
            layout=widgets.Layout(width=width, height='34px', margin='2px'),
            style=widgets.ButtonStyle(font_weight='bold', font_size='14px')
        )
        return btn
    
    # Function to update input text and perform real-time spell checking
    def update_input_text(text):
        text_area.value = text
        
        # Perform real-time spell checking
        spell_check_results = perform_spell_check(text)
        spell_check_display.value = spell_check_results
    
    # Function to perform spell checking and return HTML display
    def perform_spell_check(text):
        if not text.strip():
            return '<div id="spell-check-display" style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 50px;">Spell check results will appear here as you type...</div>'
        
        # Tokenize text to get words
        words = re.findall(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]+', text)
        words = [word.strip() for word in words if word.strip()]
        
        if not words:
            return '<div id="spell-check-display" style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 50px;">No Tigrigna words detected yet...</div>'
        
        # Check each word
        html = '<div id="spell-check-display" style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 50px;">'        
        html += '<h4 style="margin-top: 0;">Real-time Spell Check:</h4>'
        html += '<div style="display: flex; flex-wrap: wrap; gap: 10px;">'
        
        for word in words:
            is_correct = word in dictionary
            
            if is_correct:
                html += f'<div style="padding: 5px; border-radius: 4px; background-color: #e8f5e9; color: green;">{word} ✓</div>'
            else:
                suggestions = get_suggestions(word, dictionary)
                suggestion_text = ', '.join(suggestions[:3]) if suggestions else 'No suggestions'
                html += f'<div style="padding: 5px; border-radius: 4px; background-color: #ffebee; color: red;" title="Suggestions: {suggestion_text}">{word} ✗</div>'
        
        html += '</div></div>'
        return html
    
    # Function to show vowel variants when a character is clicked
    def show_variants(base_char):
        current_base[0] = base_char
        variants = generate_variants(base_char)
        
        # Create buttons for each variant with green background
        variant_buttons = []
        for i, variant in enumerate(variants[:3]):  # Show first 3 variants in top row
            btn = widgets.Button(
                description=variant,
                layout=widgets.Layout(width='33%', height='34px', margin='2px'),
                style=widgets.ButtonStyle(
                    button_color='#4CAF50',  # Green color
                    font_weight='bold',
                    font_size='14px'
                )
            )
            
            # Add click handler for the variant button
            def on_variant_click(b, variant=variant):
                # Insert the selected variant at the cursor position
                cursor_pos = text_area.cursor_pos
                text = text_area.value
                
                # Add character at cursor position
                new_text = text[:cursor_pos] + variant + text[cursor_pos:]
                update_input_text(new_text)
                
                # Move cursor after the inserted character
                text_area.cursor_pos = cursor_pos + len(variant)
            
            btn.on_click(on_variant_click)
            variant_buttons.append(btn)
        
        # Update the variants container with the new buttons
        variants_container.children = tuple(variant_buttons)
    
    # Create rows for the keyboard with the proper layout
    keyboard_rows = []
    
    # Add the main rows from the keyboard layout
    for row in keyboard_layout:
        row_buttons = [create_char_button(char) for char in row]
        row_box = widgets.HBox(row_buttons, 
                              layout=widgets.Layout(
                                  width='100%', 
                                  justify_content='space-around',
                                  padding='2px'))
        keyboard_rows.append(row_box)
    
    # Create bottom row with special keys
    bottom_row_buttons = []
    
    # Backspace button
    backspace_btn = create_char_button('⌫')
    
    # Space button (wider)
    space_btn = widgets.Button(
        description=' ',
        layout=widgets.Layout(width='50%', height='34px', margin='2px'),
        style=widgets.ButtonStyle(font_weight='bold')
    )
    
    # Punctuation buttons
    colon_btn = create_char_button('፡')
    period_btn = create_char_button('።')
    
    # Enter button
    enter_btn = create_char_button('↵')
    
    # Add buttons to bottom row
    bottom_row_buttons = [backspace_btn, space_btn, colon_btn, period_btn, enter_btn]
    
    # Create the bottom row container
    bottom_row_box = widgets.HBox(bottom_row_buttons, 
                                 layout=widgets.Layout(
                                     width='100%', 
                                     justify_content='space-around',
                                     padding='2px'))
    
    keyboard_rows.append(bottom_row_box)
    
    # Add click handlers for character buttons
    for row_box in keyboard_rows[:-1]:  # Skip the bottom special row
        for button in row_box.children:
            def on_char_click(b):
                base_char = b.description
                show_variants(base_char)
                
                # Also insert the base character
                cursor_pos = text_area.cursor_pos
                text = text_area.value
                new_text = text[:cursor_pos] + base_char + text[cursor_pos:]
                update_input_text(new_text)
                text_area.cursor_pos = cursor_pos + len(base_char)
            
            button.on_click(on_char_click)
    
    # Add handlers for special buttons
    def on_backspace_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        
        if cursor_pos > 0:
            # Delete the character before the cursor
            new_text = text[:cursor_pos-1] + text[cursor_pos:]
            update_input_text(new_text)
            text_area.cursor_pos = cursor_pos - 1
    
    def on_space_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + ' ' + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + 1
    
    def on_punctuation_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + b.description + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + len(b.description)
    
    def on_enter_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + '\n' + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + 1
    
    # Assign handlers to special buttons
    backspace_btn.on_click(on_backspace_click)
    space_btn.on_click(on_space_click)
    colon_btn.on_click(on_punctuation_click)
    period_btn.on_click(on_punctuation_click)
    enter_btn.on_click(on_enter_click)
    
    # Add observer for direct text entry in text area
    def on_text_change(change):
        if change['name'] == 'value':
            # Update the spell check display when text changes
            spell_check_results = perform_spell_check(change['new'])
            spell_check_display.value = spell_check_results
    
    text_area.observe(on_text_change, names='value')
    
    # Assemble the keyboard
    # Start with variants container (for showing the green variant buttons)
    keyboard_container.children = [variants_container]
    
    # Add the main keyboard rows
    keyboard_container.children = list(keyboard_container.children) + keyboard_rows
    
    # Add the keyboard label at the bottom
    keyboard_container.children = list(keyboard_container.children) + [keyboard_label]
    
    # Create the complete UI with text area, spell check display, and keyboard
    complete_ui = widgets.VBox([
        widgets.HTML('<h1 style="color: #4CAF50;">ትግርኛ - Tigrigna Real-time Spell Checker</h1>'),
        widgets.HTML('<p>Type in the text box below or use the keyboard. Spell checking happens automatically as you type!</p>'),
        text_area,
        spell_check_display,
        keyboard_container
    ])
    
    return complete_ui

## 4. Create and Display the Keyboard

In [17]:
# Create and display the real-time keyboard
keyboard_ui = create_real_time_keyboard()
display(keyboard_ui)

## 5. Dictionary Management Tools

Add tools to manage your Tigrigna dictionary - add new words and clear incorrect words.

In [18]:
def create_dictionary_management_ui():
    """Create UI for managing the dictionary"""
    word_input = widgets.Text(
        placeholder='Enter a Tigrigna word',
        description='New Word:',
        layout=widgets.Layout(width='300px')
    )
    
    add_button = widgets.Button(
        description='Add to Dictionary',
        button_style='success',
        icon='plus',
        layout=widgets.Layout(width='150px')
    )
    
    output = widgets.Output()
    
    def add_word_to_dictionary(word, dictionary_file='tigrigna_dictionary.txt'):
        """Add a new word to the dictionary"""
        global dictionary
        if not word or not word.strip():
            return "Please enter a valid word."
        
        # Check if the word contains Tigrigna characters
        if not re.search(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]', word):
            return "The word must contain Tigrigna characters."
        
        try:
            if word in dictionary:
                return f"The word '{word}' is already in the dictionary."
            
            # Add to in-memory dictionary
            dictionary.add(word)
            
            # Add to file
            with open(dictionary_file, 'a', encoding='utf-8') as file:
                file.write(f"\n{word}")
            
            return f"Successfully added '{word}' to the dictionary!"
        except Exception as e:
            return f"Error adding word: {e}"
    
    def on_add_button_click(b):
        word = word_input.value.strip()
        
        with output:
            output.clear_output()
            result = add_word_to_dictionary(word)
            print(result)
            if "Successfully" in result:
                # Clear the input field after successful addition
                word_input.value = ''
    
    add_button.on_click(on_add_button_click)
    
    # Create stats section
    stats_output = widgets.HTML()
    
    def update_stats():
        """Update dictionary statistics"""
        html = f"<h4>Dictionary Stats:</h4>"
        html += f"<p>Total words: {len(dictionary)}</p>"
        
        # Calculate word length distribution
        if dictionary:
            lengths = [len(word) for word in dictionary]
            avg_length = sum(lengths) / len(lengths)
            html += f"<p>Average word length: {avg_length:.1f} characters</p>"
        
        stats_output.value = html
    
    # Initial stats update
    update_stats()
    
    refresh_button = widgets.Button(
        description='Refresh Stats',
        button_style='info',
        icon='refresh',
        layout=widgets.Layout(width='120px')
    )
    
    refresh_button.on_click(lambda b: update_stats())
    
    ui = widgets.VBox([
        widgets.HTML('<h3>Dictionary Management</h3>'),
        widgets.HBox([word_input, add_button]),
        output,
        widgets.HBox([stats_output, refresh_button])
    ], layout=widgets.Layout(margin='20px 0px', padding='10px', border='1px solid #ddd'))
    
    return ui

# Create and display the dictionary management UI
dictionary_ui = create_dictionary_management_ui()
display(dictionary_ui)